In [1]:
from summarynb import show, indexed_csv, table, chunks
from malid.external.summarynb_extras import plaintext, empty
from malid import config, helpers, logger
import pandas as pd
from IPython.display import display, Markdown
from malid.trained_model_wrappers import ConvergentClusterClassifier

# Convergent sequence cluster classifiers


### Distance thresholds

#### Clustering training set:

In [2]:
config.sequence_identity_thresholds.cluster_amino_acids_across_patients

{<GeneLocus.BCR: 1>: 0.85, <GeneLocus.TCR: 2>: 0.9}

#### Assigning test sequences to clusters:

In [3]:
config.sequence_identity_thresholds.assign_test_sequences_to_clusters

{<GeneLocus.BCR: 1>: 0.85, <GeneLocus.TCR: 2>: 0.9}

In [4]:
# fold_label_train, fold_label_validation = config.get_fold_split_labels()

# Set training and held-out fold names as in scripts/train_convergent_clustering_models.py:
fold_label_train = "train_smaller1"  # Train on portion of train_smaller used to cluster sequences and train patient-level (really, specimen-level) classifier based on cluster hits
fold_label_validation = "train_smaller2"  # Evaluate on portion of train_smaller used to choose best p-value threshold for cluster association with disease, to prune cluster list

In [5]:
for gene_locus in config.gene_loci_used:
    # map target_obs_column to results_output_prefix
    targets = {}
    for target in config.classification_targets:
        targets[target] = (
            ConvergentClusterClassifier._get_output_base_dir(
                gene_locus=gene_locus,
                target_obs_column=target,
            )
            / f"{fold_label_train}_model"
        )

    print(targets)

    for target_obs_column, results_output_prefix in targets.items():
        display(
            Markdown(
                f"# {gene_locus}, {target_obs_column} trained on {fold_label_train} set"
            )
        )

        display(Markdown(f"## Specimen predictions on {fold_label_validation} set"))
        try:
            ## All results in a table
            all_results = pd.read_csv(
                f"{results_output_prefix}.compare_model_scores.tsv",
                sep="\t",
                index_col=0,
            )
            show(table(all_results), headers=["All results, sorted"])

            models_of_interest = all_results.index

            ## Confusion matrices
            for model_names in chunks(models_of_interest, 4):
                show(
                    [
                        [
                            plaintext(
                                f"{results_output_prefix}.classification_report.{model_name}.txt"
                            )
                            for model_name in model_names
                        ],
                        [
                            f"{results_output_prefix}.confusion_matrix.{model_name}.png"
                            for model_name in model_names
                        ],
                    ],
                    headers=model_names,
                    max_width=500,
                )
        except FileNotFoundError as err:
            logger.warning(f"Not run: {err}")

        display(
            Markdown(
                f"## Apply {fold_label_train} model -- Validation set performance - With and without tuning on validation set"
            )
        )
        try:
            ## All results in a table
            all_results = pd.read_csv(
                f"{results_output_prefix}.compare_model_scores.validation_set_performance.tsv",
                sep="\t",
                index_col=0,
            )
            show(table(all_results), headers=["All results, sorted"])

            models_of_interest = all_results.index

            ## Confusion matrices
            for model_names in chunks(models_of_interest, 4):
                show(
                    [
                        [
                            plaintext(
                                f"{results_output_prefix}.validation_set_performance.{model_name}.classification_report.txt"
                            )
                            for model_name in model_names
                        ],
                        [
                            f"{results_output_prefix}.validation_set_performance.{model_name}.confusion_matrix.png"
                            for model_name in model_names
                        ],
                    ],
                    max_width=500,
                    headers=model_names,
                )
        except FileNotFoundError as err:
            logger.warning(f"Not run: {err}")

        display(
            Markdown(
                f"## Apply {fold_label_train} model -- Test set performance - With and without tuning on validation set"
            )
        )
        try:
            ## All results in a table
            all_results = pd.read_csv(
                f"{results_output_prefix}.compare_model_scores.test_set_performance.tsv",
                sep="\t",
                index_col=0,
            )
            show(table(all_results), headers=["All results, sorted"])

            models_of_interest = all_results.index

            ## Confusion matrices
            for model_names in chunks(models_of_interest, 4):
                show(
                    [
                        [
                            plaintext(
                                f"{results_output_prefix}.test_set_performance.{model_name}.classification_report.txt"
                            )
                            for model_name in model_names
                        ],
                        [
                            f"{results_output_prefix}.test_set_performance.{model_name}.confusion_matrix.png"
                            for model_name in model_names
                        ],
                    ],
                    max_width=500,
                    headers=model_names,
                )
        except FileNotFoundError as err:
            logger.warning(f"Not run: {err}")

        display(Markdown("---"))

{<TargetObsColumnEnum.disease: TargetObsColumn(obs_column_name='disease', is_target_binary_for_repertoire_composition_classifier=False, available_for_cross_validation_split_strategies={<CrossValidationSplitStrategy.in_house_peak_disease_leave_one_cohort_out: CrossValidationSplitStrategyValue(data_sources_keep=[<DataSource.in_house: 1>], subtypes_keep=['Covid19 - Sero-positive (ICU)', 'Covid19 - Sero-positive (Admit)', 'Covid19 - Acute 2', 'Covid19 - Admit', 'Covid19 - ICU', 'Influenza vaccine 2021 - day 7'], diseases_to_keep_all_subtypes=['Healthy/Background', 'HIV', 'Lupus', 'T1D'], filter_specimens_func_by_study_name={'Covid19-buffycoat': <function acute_disease_choose_most_peak_timepoint at 0x7fdd2808b9d0>, 'Covid19-Stanford': <function acute_disease_choose_most_peak_timepoint at 0x7fdd2808b9d0>}, gene_loci_supported=<GeneLocus.BCR|TCR: 3>, exclude_study_names=['IBD pre-pandemic Yoni'], study_names_for_held_out_set=['Covid19-buffycoat', 'Healthy-StanfordBloodCenter_included-in-reseq

# GeneLocus.BCR, TargetObsColumnEnum.disease trained on train_smaller1 set

## Specimen predictions on train_smaller2 set

linearsvm_ovr,lasso_cv_lambda1se,elasticnet_cv0.75_lambda1se,ridge_cv
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.901 +/- 0.004 (in 3 folds) ROC-AUC (macro OvO): 0.907 +/- 0.005 (in 3 folds) au-PRC (weighted OvO): 0.912 +/- 0.006 (in 3 folds) au-PRC (macro OvO): 0.919 +/- 0.006 (in 3 folds) Accuracy: 0.666 +/- 0.005 (in 3 folds) MCC: 0.576 +/- 0.003 (in 3 folds) Global scores without abstention: Accuracy: 0.665 MCC: 0.575 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.663 +/- 0.009 (in 3 folds) MCC: 0.573 +/- 0.001 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention: Accuracy: 0.663 MCC: 0.573 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.83 0.69 0.75 29 HIV 0.90 0.80 0.85 45 Healthy/Background 0.73 0.71 0.72 103 Influenza 0.00 0.00 0.00 17 Lupus 0.72 0.56 0.63 41 T1D 0.40 0.75 0.52 44 Unknown 0.00 0.00 0.00 0 accuracy 0.66 279 macro avg 0.51 0.50 0.50 279 weighted avg 0.67 0.66 0.66 279,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.888 +/- 0.011 (in 3 folds) ROC-AUC (macro OvO): 0.887 +/- 0.013 (in 3 folds) au-PRC (weighted OvO): 0.896 +/- 0.018 (in 3 folds) au-PRC (macro OvO): 0.898 +/- 0.017 (in 3 folds) Accuracy: 0.662 +/- 0.028 (in 3 folds) MCC: 0.577 +/- 0.030 (in 3 folds) Global scores without abstention: Accuracy: 0.662 MCC: 0.576 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.660 +/- 0.031 (in 3 folds) MCC: 0.575 +/- 0.033 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention: Accuracy: 0.659 MCC: 0.574 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.83 0.66 0.73 29 HIV 0.92 0.80 0.86 45 Healthy/Background 0.75 0.68 0.71 103 Influenza 0.00 0.00 0.00 17 Lupus 0.66 0.56 0.61 41 T1D 0.41 0.82 0.55 44 Unknown 0.00 0.00 0.00 0 accuracy 0.66 279 macro avg 0.51 0.50 0.49 279 weighted avg 0.67 0.66 0.65 279,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.885 +/- 0.010 (in 3 folds) ROC-AUC (macro OvO): 0.885 +/- 0.009 (in 3 folds) au-PRC (weighted OvO): 0.902 +/- 0.010 (in 3 folds) au-PRC (macro OvO): 0.905 +/- 0.006 (in 3 folds) Accuracy: 0.644 +/- 0.010 (in 3 folds) MCC: 0.550 +/- 0.016 (in 3 folds) Global scores without abstention: Accuracy: 0.644 MCC: 0.550 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.642 +/- 0.013 (in 3 folds) MCC: 0.547 +/- 0.015 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention: Accuracy: 0.642 MCC: 0.548 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.82 0.62 0.71 29 HIV 0.94 0.76 0.84 45 Healthy/Background 0.71 0.69 0.70 103 Influenza 0.00 0.00 0.00 17 Lupus 0.68 0.51 0.58 41 T1D 0.39 0.80 0.53 44 Unknown 0.00 0.00 0.00 0 accuracy 0.64 279 macro avg 0.51 0.48 0.48 279 weighted avg 0.66 0.64 0.64 279,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.883 +/- 0.019 (in 3 folds) ROC-AUC (macro OvO): 0.892 +/- 0.016 (in 3 folds) au-PRC (weighted OvO): 0.891 +/- 0.026 (in 3 folds) au-PRC (macro OvO): 0.902 +/- 0.020 (in 3 folds) Accuracy: 0.582 +/- 0.021 (in 3 folds) MCC: 0.518 +/- 0.003 (in 3 folds) Global scores without abstention: Accuracy: 0.582 MCC: 0.512 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.570 +/- 0.027 (in 3 folds) MCC: 0.500 +/- 0.008 (in 3 folds) Unknown/abstention proportion: 0.022 +/- 0.011 (in 3 folds) Global scores with abstention: Accuracy: 0.570 MCC: 0.495 Unknown/abstention proportion: 0.022 Abstention label: Un

ridge_sklearn_with_lambdamax,ridge_cv_lambda1se,elasticnet_cv_lambda1se,lasso_sklearn_with_lambdamax
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.883 +/- 0.018 (in 3 folds) ROC-AUC (macro OvO): 0.892 +/- 0.016 (in 3 folds) au-PRC (weighted OvO): 0.891 +/- 0.026 (in 3 folds) au-PRC (macro OvO): 0.902 +/- 0.020 (in 3 folds) Accuracy: 0.582 +/- 0.021 (in 3 folds) MCC: 0.518 +/- 0.003 (in 3 folds) Global scores without abstention: Accuracy: 0.582 MCC: 0.512 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.570 +/- 0.027 (in 3 folds) MCC: 0.500 +/- 0.008 (in 3 folds) Unknown/abstention proportion: 0.022 +/- 0.011 (in 3 folds) Global scores with abstention: Accuracy: 0.570 MCC: 0.495 Unknown/abstention proportion: 0.022 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 1.00 0.48 0.65 29 HIV 1.00 0.64 0.78 45 Healthy/Background 0.73 0.51 0.60 103 Influenza 0.86 0.35 0.50 17 Lupus 0.74 0.41 0.53 41 T1D 0.31 0.91 0.47 44 Unknown 0.00 0.00 0.00 0 accuracy 0.57 279 macro avg 0.66 0.47 0.51 279 weighted avg 0.74 0.57 0.60 279,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.883 +/- 0.017 (in 3 folds) ROC-AUC (macro OvO): 0.892 +/- 0.013 (in 3 folds) au-PRC (weighted OvO): 0.891 +/- 0.025 (in 3 folds) au-PRC (macro OvO): 0.902 +/- 0.019 (in 3 folds) Accuracy: 0.586 +/- 0.027 (in 3 folds) MCC: 0.518 +/- 0.003 (in 3 folds) Global scores without abstention: Accuracy: 0.586 MCC: 0.508 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.573 +/- 0.033 (in 3 folds) MCC: 0.500 +/- 0.008 (in 3 folds) Unknown/abstention proportion: 0.022 +/- 0.011 (in 3 folds) Global scores with abstention: Accuracy: 0.573 MCC: 0.491 Unknown/abstention proportion: 0.022 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 1.00 0.48 0.65 29 HIV 1.00 0.64 0.78 45 Healthy/Background 0.70 0.53 0.60 103 Influenza 0.86 0.35 0.50 17 Lupus 0.74 0.41 0.53 41 T1D 0.32 0.89 0.47 44 Unknown 0.00 0.00 0.00 0 accuracy 0.57 279 macro avg 0.66 0.47 0.51 279 weighted avg 0.73 0.57 0.60 279,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.883 +/- 0.007 (in 3 folds) ROC-AUC (macro OvO): 0.884 +/- 0.007 (in 3 folds) au-PRC (weighted OvO): 0.904 +/- 0.007 (in 3 folds) au-PRC (macro OvO): 0.907 +/- 0.005 (in 3 folds) Accuracy: 0.644 +/- 0.010 (in 3 folds) MCC: 0.551 +/- 0.013 (in 3 folds) Global scores without abstention: Accuracy: 0.644 MCC: 0.551 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.642 +/- 0.013 (in 3 folds) MCC: 0.549 +/- 0.013 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention: Accuracy: 0.642 MCC: 0.549 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.82 0.62 0.71 29 HIV 0.94 0.76 0.84 45 Healthy/Background 0.71 0.69 0.70 103 Influenza 0.00 0.00 0.00 17 Lupus 0.75 0.51 0.61 41 T1D 0.38 0.80 0.51 44 Unknown 0.00 0.00 0.00 0 accuracy 0.64 279 macro avg 0.51 0.48 0.48 279 weighted avg 0.67 0.64 0.64 279,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.882 +/- 0.023 (in 3 folds) ROC-AUC (macro OvO): 0.880 +/- 0.025 (in 3 folds) au-PRC (weighted OvO): 0.891 +/- 0.028 (in 3 folds) au-PRC (macro OvO): 0.892 +/- 0.028 (in 3 folds) Accuracy: 0.644 +/- 0.021 (in 3 folds) MCC: 0.549 +/- 0.031 (in 3 folds) Global scores without abstention: Accuracy: 0.644 MCC: 0.550 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.642 +/- 0.022 (in 3 folds) MCC: 0.547 +/- 0.031 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention: Accuracy: 0.642 MCC: 0.547 Unknown/abstention proport

...

elasticnet_cv,elasticnet_cv.decision_thresholds_tuned,logisticregression_unregularized,logisticregression_unregularized.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.804 +/- 0.015 (in 3 folds) ROC-AUC (macro OvO): 0.807 +/- 0.021 (in 3 folds) au-PRC (weighted OvO): 0.802 +/- 0.021 (in 3 folds) au-PRC (macro OvO): 0.812 +/- 0.024 (in 3 folds) Accuracy: 0.491 +/- 0.016 (in 3 folds) MCC: 0.366 +/- 0.024 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.491 MCC: 0.365 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.489 +/- 0.014 (in 3 folds) MCC: 0.365 +/- 0.023 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.489 MCC: 0.363 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.38 0.53 0.44 58 HIV 0.40 0.23 0.30 98 Healthy/Background 0.67 0.54 0.60 197 Influenza 0.71 0.54 0.62 37 Lupus 0.53 0.53 0.53 64 T1D 0.34 0.56 0.43 96 Unknown 0.00 0.00 0.00 0 accuracy 0.49 550 macro avg 0.43 0.42 0.42 550 weighted avg 0.52 0.49 0.49 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.804 +/- 0.015 (in 3 folds) ROC-AUC (macro OvO): 0.807 +/- 0.021 (in 3 folds) au-PRC (weighted OvO): 0.802 +/- 0.021 (in 3 folds) au-PRC (macro OvO): 0.812 +/- 0.024 (in 3 folds) Accuracy: 0.480 +/- 0.028 (in 3 folds) MCC: 0.353 +/- 0.014 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.480 MCC: 0.347 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.478 +/- 0.030 (in 3 folds) MCC: 0.351 +/- 0.017 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.478 MCC: 0.345 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.40 0.59 0.48 58 HIV 0.45 0.20 0.28 98 Healthy/Background 0.62 0.55 0.58 197 Influenza 0.86 0.49 0.62 37 Lupus 0.55 0.50 0.52 64 T1D 0.31 0.53 0.39 96 Unknown 0.00 0.00 0.00 0 accuracy 0.48 550 macro avg 0.46 0.41 0.41 550 weighted avg 0.52 0.48 0.48 550,Per-fold scores: ROC-AUC (weighted OvO): 0.804 +/- 0.012 (in 3 folds) ROC-AUC (macro OvO): 0.803 +/- 0.017 (in 3 folds) au-PRC (weighted OvO): 0.806 +/- 0.016 (in 3 folds) au-PRC (macro OvO): 0.812 +/- 0.022 (in 3 folds) Accuracy: 0.527 +/- 0.024 (in 3 folds) MCC: 0.395 +/- 0.035 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.527 MCC: 0.394 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.48 0.50 0.49 58 HIV 0.61 0.49 0.54 98 Healthy/Background 0.62 0.63 0.63 197 Influenza 0.52 0.38 0.44 37 Lupus 0.49 0.58 0.53 64 T1D 0.35 0.39 0.37 96 accuracy 0.53 550 macro avg 0.51 0.49 0.50 550 weighted avg 0.53 0.53 0.53 550,Per-fold scores: ROC-AUC (weighted OvO): 0.804 +/- 0.012 (in 3 folds) ROC-AUC (macro OvO): 0.803 +/- 0.017 (in 3 folds) au-PRC (weighted OvO): 0.806 +/- 0.016 (in 3 folds) au-PRC (macro OvO): 0.812 +/- 0.022 (in 3 folds) Accuracy: 0.544 +/- 0.014 (in 3 folds) MCC: 0.421 +/- 0.021 (in 3 folds) Global scores using column name disease.separate_past_exposures: A

elasticnet_cv0.75,elasticnet_cv0.75.decision_thresholds_tuned,ridge_cv,ridge_cv.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.803 +/- 0.016 (in 3 folds) ROC-AUC (macro OvO): 0.807 +/- 0.022 (in 3 folds) au-PRC (weighted OvO): 0.801 +/- 0.022 (in 3 folds) au-PRC (macro OvO): 0.812 +/- 0.025 (in 3 folds) Accuracy: 0.487 +/- 0.008 (in 3 folds) MCC: 0.361 +/- 0.017 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.487 MCC: 0.359 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.486 +/- 0.008 (in 3 folds) MCC: 0.359 +/- 0.017 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.485 MCC: 0.358 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.39 0.52 0.44 58 HIV 0.40 0.24 0.30 98 Healthy/Background 0.67 0.54 0.60 197 Influenza 0.74 0.54 0.62 37 Lupus 0.53 0.56 0.55 64 T1D 0.32 0.53 0.40 96 Unknown 0.00 0.00 0.00 0 accuracy 0.49 550 macro avg 0.44 0.42 0.42 550 weighted avg 0.52 0.49 0.49 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.803 +/- 0.016 (in 3 folds) ROC-AUC (macro OvO): 0.807 +/- 0.022 (in 3 folds) au-PRC (weighted OvO): 0.801 +/- 0.022 (in 3 folds) au-PRC (macro OvO): 0.812 +/- 0.025 (in 3 folds) Accuracy: 0.487 +/- 0.036 (in 3 folds) MCC: 0.353 +/- 0.030 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.487 MCC: 0.346 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.486 +/- 0.038 (in 3 folds) MCC: 0.351 +/- 0.032 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.006 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.485 MCC: 0.344 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.42 0.55 0.47 58 HIV 0.43 0.24 0.31 98 Healthy/Background 0.61 0.60 0.61 197 Influenza 0.71 0.46 0.56 37 Lupus 0.54 0.50 0.52 64 T1D 0.31 0.45 0.37 96 Unknown 0.00 0.00 0.00 0 accuracy 0.49 550 macro avg 0.43 0.40 0.41 550 weighted avg 0.50 0.49 0.49 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.796 +/- 0.019 (in 3 folds) ROC-AUC (macro OvO): 0.799 +/- 0.025 (in 3 folds) au-PRC (weighted OvO): 0.782 +/- 0.023 (in 3 folds) au-PRC (macro OvO): 0.794 +/- 0.027 (in 3 folds) Accuracy: 0.462 +/- 0.005 (in 3 folds) MCC: 0.354 +/- 0.023 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.462 MCC: 0.347 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.458 +/- 0.003 (in 3 folds) MCC: 0.349 +/- 0.019 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.013 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.458 MCC: 0.343 Unknown/abstention proportion: 0.007 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.24 0.60 0.35 58 HIV 0.45 0.30 0.36 98 Healthy/Background 0.74 0.48 0.58 197 Influenza 0.82 0.49 0.61 37 Lupus 

rf_multiclass,rf_multiclass.decision_thresholds_tuned,elasticnet_cv_each_disease_versus_healthy,elasticnet_cv_each_disease_versus_healthy.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.795 +/- 0.029 (in 3 folds) ROC-AUC (macro OvO): 0.796 +/- 0.033 (in 3 folds) au-PRC (weighted OvO): 0.794 +/- 0.018 (in 3 folds) au-PRC (macro OvO): 0.801 +/- 0.023 (in 3 folds) Accuracy: 0.540 +/- 0.017 (in 3 folds) MCC: 0.406 +/- 0.023 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.540 MCC: 0.404 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.536 +/- 0.012 (in 3 folds) MCC: 0.402 +/- 0.017 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.013 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.536 MCC: 0.400 Unknown/abstention proportion: 0.007 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.38 0.34 0.36 58 HIV 0.49 0.39 0.43 98 Healthy/Background 0.63 0.69 0.66 197 Influenza 0.78 0.57 0.66 37 Lupus 0.63 0.50 0.56 64 T1D 0.40 0.50 0.44 96 Unknown 0.00 0.00 0.00 0 accuracy 0.54 550 macro avg 0.47 0.43 0.44 550 weighted avg 0.55 0.54 0.54 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.795 +/- 0.029 (in 3 folds) ROC-AUC (macro OvO): 0.796 +/- 0.033 (in 3 folds) au-PRC (weighted OvO): 0.794 +/- 0.018 (in 3 folds) au-PRC (macro OvO): 0.801 +/- 0.023 (in 3 folds) Accuracy: 0.524 +/- 0.022 (in 3 folds) MCC: 0.389 +/- 0.019 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.524 MCC: 0.378 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.520 +/- 0.025 (in 3 folds) MCC: 0.386 +/- 0.023 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.013 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.520 MCC: 0.375 Unknown/abstention proportion: 0.007 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.34 0.43 0.38 58 HIV 0.49 0.38 0.43 98 Healthy/Background 0.59 0.72 0.65 197 Influenza 0.83 0.51 0.63 37 Lupus 0.61 0.48 0.54 64 T1D 0.39 0.33 0.36 96 Unknown 0.00 0.00 0.00 0 accuracy 0.52 550 macro avg 0.46 0.41 0.43 550 weighted avg 0.53 0.52 0.52 550,Per-fold scores: ROC-AUC (weighted OvO): 0.733 +/- 0.024 (in 3 folds) ROC-AUC (macro OvO): 0.752 +/- 0.031 (in 3 folds) au-PRC (weighted OvO): 0.767 +/- 0.029 (in 3 folds) au-PRC (macro OvO): 0.782 +/- 0.039 (in 3 folds) Accuracy: 0.308 +/- 0.044 (in 3 folds) MCC: 0.242 +/- 0.047 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.307 MCC: 0.220 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.43 0.26 0.32 58 HIV 0.65 0.58 0.61 98 Healthy/Background 0.00 0.00 0.00 197 Influenza 0.32 0.16 0.21 37 Lupus 0.21 0.69 0.32 64 T1D 0.23 0.49 0.32 96 accuracy 0.31 550 macro avg 0.31 0.36 0.30 550 weighted avg 0.25 0.31 0.25 550,Per-fold scores: ROC-AUC (weighted OvO): 0.733 +/- 0.024 (in 3 folds) ROC-AUC (macro OvO): 0.752 +/- 0.031 (in 3 folds) au-PRC (weighted OvO): 0.767 +/- 0.029 (in 3 folds) au-PRC (macro OvO): 0.782 +/- 0.039 (in 3 folds) Accuracy: 0.370 +/- 0.071 (in 3 folds) MCC: 0.317 +/- 0.091 (in 3 folds) Global scores using column name disease.separate

dummy_stratified.decision_thresholds_tuned,dummy_stratified,dummy_most_frequent.decision_thresholds_tuned,dummy_most_frequent
Per-fold scores: ROC-AUC (weighted OvO): 0.508 +/- 0.008 (in 3 folds) ROC-AUC (macro OvO): 0.509 +/- 0.014 (in 3 folds) au-PRC (weighted OvO): 0.512 +/- 0.007 (in 3 folds) au-PRC (macro OvO): 0.513 +/- 0.011 (in 3 folds) Accuracy: 0.209 +/- 0.007 (in 3 folds) MCC: 0.013 +/- 0.007 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.209 MCC: 0.013 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.17 0.22 0.19 58 HIV 0.20 0.20 0.20 98 Healthy/Background 0.36 0.27 0.31 197 Influenza 0.02 0.03 0.02 37 Lupus 0.21 0.25 0.23 64 T1D 0.11 0.11 0.11 96 accuracy 0.21 550 macro avg 0.18 0.18 0.18 550 weighted avg 0.23 0.21 0.22 550,Per-fold scores: ROC-AUC (weighted OvO): 0.508 +/- 0.008 (in 3 folds) ROC-AUC (macro OvO): 0.509 +/- 0.014 (in 3 folds) au-PRC (weighted OvO): 0.512 +/- 0.007 (in 3 folds) au-PRC (macro OvO): 0.513 +/- 0.011 (in 3 folds) Accuracy: 0.209 +/- 0.007 (in 3 folds) MCC: 0.013 +/- 0.007 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.209 MCC: 0.013 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.17 0.22 0.19 58 HIV 0.20 0.20 0.20 98 Healthy/Background 0.36 0.27 0.31 197 Influenza 0.02 0.03 0.02 37 Lupus 0.21 0.25 0.23 64 T1D 0.11 0.11 0.11 96 accuracy 0.21 550 macro avg 0.18 0.18 0.18 550 weighted avg 0.23 0.21 0.22 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.500 +/- 0.000 (in 3 folds) ROC-AUC (macro OvO): 0.500 +/- 0.000 (in 3 folds) au-PRC (weighted OvO): 0.500 +/- 0.000 (in 3 folds) au-PRC (macro OvO): 0.500 +/- 0.000 (in 3 folds) Accuracy: 0.361 +/- 0.006 (in 3 folds) MCC: 0.000 +/- 0.000 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.361 MCC: 0.000 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.358 +/- 0.003 (in 3 folds) MCC: 0.014 +/- 0.025 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.013 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.358 MCC: 0.025 Unknown/abstention proportion: 0.007 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.00 0.00 0.00 58 HIV 0.00 0.00 0.00 98 Healthy/Background 0.36 1.00 0.53 197 Influenza 0.00 0.00 0.00 37 Lupus 0.00 0.00 0.00 64 T1D 0.00 0.00 0.00 96 Unknown 0.00 0.00 0.00 0 accuracy 0.36 550 macro avg 0.05 0.14 0.08 550 weighted avg 0.13 0.36 0.19 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.500 +/- 0.000 (in 3 folds) ROC-AUC (macro OvO): 0.500 +/- 0.000 (in 3 folds) au-PRC (weighted OvO): 0.500 +/- 0.000 (in 3 folds) au-PRC (macro OvO): 0.500 +/- 0.000 (in 3 folds) Accuracy: 0.361 +/- 0.006 (in 3 folds) MCC: 0.000 +/- 0.000 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.361 MCC: 0.000 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.358 +/- 0.003 (in 3 folds) MCC: 0.014 +/- 0.025 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.013 (in 3 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.358 MCC: 0.025 Unknown/abstention proportion: 0.007 Abstention label: Unknown Global evaluation column name: dise

---